In [1]:
expt_configs = '/home/ildefonso/Documents/repositories/coding/segmentation/compare-seg-models/experiment_configs'
net_configs = '/home/ildefonso/Documents/repositories/coding/segmentation/compare-seg-models/network_configs/'

In [3]:
import os
os.listdir(expt_configs)

['config_02282018_koumura_bird0_GPU1.ini']

In [2]:
import seg_nets

/home/ildefonso/anaconda3/envs/tcn/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
../learn_curve_keras.py

SyntaxError: invalid syntax (<ipython-input-4-f0b79f9bc905>, line 1)

In [5]:
ls

initial-experiment.ipynb


In [8]:
cd /home/ildefonso/Documents/repositories/coding/segmentation/compare-seg-models

/home/ildefonso/Documents/repositories/coding/segmentation/compare-seg-models


In [15]:
!python make_data.py ./experiment_configs/config_20180406_koumura_bird0_GPU1.ini

/home/ildefonso/anaconda3/envs/tcn/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/112.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/24.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/95.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/38.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/131.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/86.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/80.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/85.wav
making

making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/130.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/118.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/132.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/22.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/45.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/60.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/55.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/2.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/78.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/74.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/17.wav
making .spect file for /home/ildefonso/data/birdsong_recognition/Bird0/Wave/110.wav
m

In [24]:
%run learn_curve_keras.py ./experiment_configs/config_20180406_koumura_bird0_GPU1.ini

ImportError: cannot import name 'reshape_inputs_and_make_masks'

In [4]:
from math import ceil

import numpy as np

from keras.models import Model
from keras.layers import Input, TimeDistributed, Add, Multiply
from keras.layers.core import Activation, SpatialDropout1D, Lambda, Reshape, Dense
from keras.layers.convolutional import Conv1D, Conv2D, ZeroPadding1D, Cropping1D
from keras.layers.convolutional import MaxPooling1D, MaxPooling2D, UpSampling1D
from keras.layers import Bidirectional
from keras.layers.recurrent import LSTM

import tensorflow as tf
from keras import backend as K

from keras.activations import relu
from functools import partial

In [61]:
x.get_shape()[0]

Dimension(None)

In [ ]:
K.map_fn(lambda i, time_steps: , dtype=tf.int32)

In [12]:
n_filters_by_layer = (32, 64)
max_len = 19292
num_feat = 152
inputs = Input(shape=(max_len, num_feat))
time_steps = 88
n_classes = 13


x = inputs
x = Conv1D(filters=1,kernel_size=time_steps,strides=1,padding='same',activation='linear')(x)
# for n_filters in n_filters_by_layer:
#     x = Conv2D(n_filters, kernel_size=(5, 5),
#                padding="same", activation='relu')(x)
#     x = MaxPooling2D(pool_size=(1, 8),
#                      strides=(1, 8))(x)
# conv_out_shape = x.get_shape().as_list()
# num_hidden = conv_out_shape[2] * conv_out_shape[3]
# new_shape = (-1, num_hidden)
# x = Reshape(new_shape)(x)
# x = Bidirectional(
#     LSTM(num_hidden, return_sequences=True, dropout=0.25,
#          recurrent_dropout=0.1))(x)
# x = TimeDistributed(Dense(n_classes, activation="softmax"))(x)

In [13]:
x

<tf.Tensor 'conv1d_2/add:0' shape=(?, 19292, 1) dtype=float32>

In [6]:
model = Model(inputs=inputs, outputs=x)

TypeError: Output tensors to a Model must be Keras tensors. Found: Tensor("frame/Reshape_4:0", shape=(?, 19205, 88, 152, 1), dtype=float32)

In [95]:
model.layers[-1].output_shape

(None, 19292, 13)

In [18]:
def out_width(in_width, filter_width, stride):
    return ceil(float(in_width - filter_width + 1) / float(stride))

In [25]:
freq_bins_after_pool1 = out_width(152,
                                  8,
                                  8)
freq_bins_after_pool2 = out_width(freq_bins_after_pool1,
                                  8,
                                  8)

In [14]:
from numpy.lib import stride_tricks

In [15]:
import joblib

In [20]:
cd /home/ildefonso/data/birdsong_recognition/Bird1/Wave/spectrograms_180407_204020/

/home/ildefonso/data/birdsong_recognition/Bird1/Wave/spectrograms_180407_204020


In [21]:
trd = joblib.load('train_data_dict')

In [23]:
X_tr = trd['X_train']

In [24]:
X_tr.shape

(152, 300000)

In [28]:
t = stride_tricks.as_strided(X_tr, shape=(X_tr.shape[-1]-88,152,88), strides=(X_tr.strides)

ValueError: mismatch in length of strides and shape

In [42]:
t = array_for_sliding_window(X_tr, (152,300))

In [45]:
tsq = np.squeeze(t)

In [46]:
tsq.shape

(299701, 152, 300)

In [31]:
import numpy as np

def array_for_sliding_window(x, wshape):
    """Build a sliding-window representation of x.

    The last dimension(s) of the output array contain the data of
    the specific window.  The number of dimensions in the output is 
    twice that of the input.

    Parameters
    ----------
    x : ndarray_like
       An array for which is desired a representation to which sliding-windows 
       computations can be easily applied.
    wshape : int or tuple
       If an integer, then it is converted into a tuple of size given by the 
       number of dimensions of x with every element set to that integer.
       If a tuple, then it should be the shape of the desired window-function

    Returns
    -------
    out : ndarray
        Return a zero-copy view of the data in x so that operations can be 
        performed over the last dimensions of this new array and be equivalent 
        to a sliding window calculation.  The shape of out is 2*x.ndim with 
        the shape of the last nd dimensions equal to wshape while the shape 
        of the first n dimensions is found by subtracting the window shape
        from the input shape and adding one in each dimension.  This is 
        the number of "complete" blocks of shape wshape in x.

    Raises
    ------
    ValueError
        If the size of wshape is not x.ndim (unless wshape is an integer).
        If one of the dimensions of wshape exceeds the input array. 

    Examples
    --------
    >>> x = np.linspace(1,5,5)
    >>> x
    array([ 1.,  2.,  3.,  4.,  5.])

    >>> array_for_rolling_window(x, 3)
    array([[ 1.,  2.,  3.],
           [ 2.,  3.,  4.],
           [ 3.,  4.,  5.]])
           
    >>> x = np.arange(1,17).reshape(4,4)
    >>> x
    array([[ 1,  2,  3,  4],
           [ 5,  6,  7,  8],
           [ 9, 10, 11, 12],
           [13, 14, 15, 16]])

    >>> array_for_rolling_window(x, 3)
    array([[[[ 1,  2,  3],
             [ 5,  6,  7],
             [ 9, 10, 11]],

            [[ 2,  3,  4],
             [ 6,  7,  8],
             [10, 11, 12]]],

           [[[ 5,  6,  7],
             [ 9, 10, 11],
             [13, 14, 15]],

            [[ 6,  7,  8],
             [10, 11, 12],
             [14, 15, 16]]]])
    """
    x = np.asarray(x)

    try:
        nd = len(wshape)
    except TypeError:
        wshape = tuple(wshape for i in x.shape)
        nd = len(wshape)
    if nd != x.ndim:
        raise ValueError("wshape has length {0} instead of "
                         "x.ndim which is {1}".format(len(wshape), x.ndim)) 
    
    out_shape = tuple(xi-wi+1 for xi, wi in zip(x.shape, wshape)) + wshape
    if not all(i>0 for i in out_shape):
        raise ValueError("wshape is bigger than input array along at "
                         "least one dimension")

    out_strides = x.strides*2
    
    return np.lib.stride_tricks.as_strided(x, out_shape, out_strides)


In [35]:
X_tr.strides*2

(2400000, 8, 2400000, 8)